# About
This notebook is to wrangle `categories` column in `yelp_team7_dataset_restaurant.json`. New columns based on category types were created: 
* `cuisine_type`
* `food_type`
* `atmosphere_type`
* `dietary_type`

These columns and `business_id` were saved in a separate file, `yelp_team7_dataset_categories.json`.

In [1]:
import json
import pandas as pd
import preprocess

/opt/anaconda3/envs/ds4aw-env/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Load restaurant dataset

In [2]:
restaurant_df = pd.read_json('yelp_team7_dataset_restaurant.json')

In [3]:
restaurant_df.shape

(14970, 14)

# Create new columns of category types

See `categories_mapping.xls` for detailed steps to generate category mapping files used in this section.

In [4]:
categories = [['categories_mapping_cuisine_type.csv', 'cuisine_type'],
              ['categories_mapping_food_type.csv', 'food_type'],
              ['categories_mapping_atmosphere_type.csv', 'atmosphere_type'],
              ['categories_mapping_dietary_type.csv', 'dietary_type']]

In [5]:
# map the categories column into four new columns of category types
for a_type in categories:
    preprocess.gen_category_column(restaurant_df, a_type[0], a_type[1])

In [6]:
restaurant_df.shape

(14970, 18)